In [1]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [3]:
df = pd.read_csv('./data/beige_book_district.csv')
df.head()

,district_report,date,district
0,\n\r\n var ref = document.referrer;\r\n ...,2019-04,at
1,\n\r\n var ref = document.referrer;\r\n ...,2019-03,at
2,\n\r\n var ref = document.referrer;\r\n ...,2019-01,at
3,\n\r\n var ref = document.referrer;\r\n ...,2018-12,at
4,\n\r\n var ref = document.referrer;\r\n ...,2018-10,at


In [7]:
# Convert to list
data = df['district_report'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:1])

<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-7-821458d70d3d>:5: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-7-821458d70d3d>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


## Tokenize and Clean-up using gensim’s simple_preprocess()

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

## Remove Stopwords

In [10]:
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
stop_words = stopwords.words('english')
stop_words.extend(['http','https','www','com','@','...','…', 'var', 'ref', 'document', 
             'referrer', 'var', 'bbpath', 'href', 'archive', 'function', 'beige',
            'book', 'federal', 'reserve', 'summary', 'indexof', 'backtoresults',
            'history', 'go', 'else', 'window', 'location', 'back', 'search',
            'this', 'report', 'prepared', 'federal', 'reserve', 'bank', 'summarizes',
            'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august',
            'september', 'october', 'november', 'december', 'atlanta', 'boston', 'chicago',
            'cleveland', 'dallas', 'kansas', 'minneapolis', 'york', 'philadelphia', 
             'richmond', 'san', 'francisco', 'st', 'louis', 'officials', 'comments', 
             'views', 'commentary', 'national', 'based', 'information', 'collected',
             'received', 'contacts', 'outside', 'system', 'page', 'url', 'link', 
                   'district', 'districts', 'activity'])

In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)


## Lemmatization

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:2])


In [82]:
from nltk.corpus import wordnet as wn
wn.synsets('percent')

[Synset('percentage.n.01')]

In [83]:
wn.synset('percentage.n.01').lemma_names()

['percentage', 'percent', 'per_centum', 'pct']

In [84]:
from nltk.corpus import wordnet as wn
wn.synsets('interest')

[Synset('interest.n.01'),
 Synset('sake.n.01'),
 Synset('interest.n.03'),
 Synset('interest.n.04'),
 Synset('interest.n.05'),
 Synset('interest.n.06'),
 Synset('pastime.n.01'),
 Synset('interest.v.01'),
 Synset('concern.v.02'),
 Synset('matter_to.v.01')]

In [87]:
wn.synset('interest.n.04').lemma_names()

['interest']

## Create the Document-Word matrix

In [22]:
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=20,
                             stop_words='english',
                             ngram_range= (2, 3)
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [23]:
len(vectorizer.get_feature_names())

23445

## Check the Sparsicity

In [24]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  1.2131088266724863 %


In [25]:
data_dense

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.04734498, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

## Build LDA model with sklearn

In [27]:
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=20190512,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=20190512, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


## Diagnose model performance with perplexity and log-likelihood

In [28]:
# Log Likelihood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -862181.282956067
Perplexity:  51977.40998285477
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'n_topics': None,
 'perp_tol': 0.1,
 'random_state': 20190512,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [29]:
# Define Search Param
params = {
    'n_components': [5, 10, 15, 20, 25, 30], 
    'learning_decay': [.5, .7, .9]
}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=params, cv=5)

# Do the Grid Search
model.fit(data_vectorized)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## Best topic model and its parameters

In [30]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 5}
Best Log Likelihood Score:  -198658.64352670324
Model Perplexity:  35862.24387198418


In [31]:
model.get_params

<bound method BaseEstimator.get_params of GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)>

In [ ]:
#vectorizer.get_feature_names()

In [46]:
best_lda_model.get_params

<bound method BaseEstimator.get_params of LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.5,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)>

## Dominant topic in each document

In [45]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, 
                                 index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.24,0.01,0.73,0.01,0.01,2
Doc1,0.29,0.01,0.68,0.01,0.01,2
Doc2,0.22,0.01,0.75,0.01,0.01,2
Doc3,0.26,0.01,0.7,0.01,0.01,2
Doc4,0.3,0.01,0.67,0.01,0.01,2
Doc5,0.3,0.01,0.67,0.01,0.01,2
Doc6,0.3,0.01,0.67,0.01,0.01,2
Doc7,0.27,0.01,0.68,0.01,0.02,2
Doc8,0.32,0.01,0.61,0.01,0.05,2
Doc9,0.43,0.01,0.54,0.01,0.01,2


### Review topics distribution across documents

In [47]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution


,Topic Num,Num Documents
0,2,4736
1,0,292
2,3,120


### Visualize the LDA model

In [48]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      -41.574631  -97.306091       1        1  83.233141
0        3.790894   31.322222       2        1   9.275161
3       87.052467 -142.672684       3        1   4.445053
4       45.422768  -55.674572       4        1   1.707397
1      132.420441  -14.044519       5        1   1.339248, topic_info=      Category        Freq                       Term       Total  loglift  \
16517  Default  131.000000                real estate  131.000000  30.0000   
12887  Default   26.000000                   new city   26.000000  29.0000   
12963  Default   20.000000                 new jersey   20.000000  28.0000   
20494  Default   22.000000               south dakota   22.000000  27.0000   
2372   Default   37.000000               compare year   37.000000  26.0000   
13161  Default   21.000000               north dakota   21.000000  25.0000   
22315  Default   36.000000               vacancy rate   36.000000  24.0000   
2280   Default   56.000000     commercial real estate   56.000000  23.0000   
2279   Default   56.000000            commercial real   56.000000  22.0000   
8391   Default   47.000000                  home sale   47.000000  21.0000   
23156  Default   50.000000                 year early   50.000000  20.0000   
9081   Default   34.000000           increase percent   34.000000  19.0000   
19534  Default   27.000000                 sell price   27.000000  18.0000   
17860  Default   20.000000       residential mortgage   20.000000  17.0000   
16558  Default   33.000000         real estate market   33.000000  16.0000   
5604   Default   33.000000              estate market   33.000000  15.0000   
2912   Default   59.000000          consumer spending   59.000000  14.0000   
10325  Default   14.000000               lending rate   14.000000  13.0000   
8496   Default   25.000000             housing market   25.000000  12.0000   
18951  Default   23.000000               sale percent   23.000000  11.0000   
17800  Default   21.000000           reporting period   21.000000  10.0000   
3629   Default   20.000000            credit standard   20.000000   9.0000   
4172   Default   12.000000           delinquency rate   12.000000   8.0000   
2833   Default   18.000000        consumer confidence   18.000000   7.0000   
2747   Default   28.000000          construction real   28.000000   6.0000   
2748   Default   28.000000   construction real estate   28.000000   5.0000   
23152  Default   25.000000               year earlier   25.000000   4.0000   
17563  Default   19.000000              report period   19.000000   3.0000   
1475   Default   28.000000         business condition   28.000000   2.0000   
16526  Default   23.000000   real estate construction   23.000000   1.0000   
...        ...         ...                        ...         ...      ...   
13871   Topic5    0.312979        outpace sale higher    0.796201   3.3793   
21965   Topic5    0.287101          trade firm report    0.733082   3.3756   
22020   Topic5    0.374110   transportation wholesale    0.956480   3.3744   
16046   Topic5    0.283317  quality unchanged compare    0.730275   3.3662   
8326    Topic5    0.278042      home continue outpace    0.724988   3.3547   
4564    Topic5    0.271033       deposit level energy    0.717319   3.3398   
5721    Topic5    0.268852        exclude credit card    0.716302   3.3331   
5720    Topic5    0.268852             exclude credit    0.716302   3.3331   
13102   Topic5    0.256590           nonauto retailer    0.708852   3.2969   
13101   Topic5    0.397094        nonauto retail sale    1.101729   3.2926   
12638   Topic5    0.248568    month professional high    0.695550   3.2841   
15320   Topic5    0.304634       price level continue    0.870874   3.2627   
10424   Topic5    0.232700        level energy energy    0.680725   3.2397   
12637   Topic5    0.254478         month 

## Topic keywords

In [81]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
topicnames = [0, 1, 2, 3, 4]
df_topic_keywords.index = topicnames

# View
df_topic_keywords

,ability pass,ability pass cost,ability raise,ability raise price,able increase,able meet,able pass,able pass cost,able pass increase,able raise,...,yield fall,yield generally,yield good,yield high,yield low,yield percent,yield prospect,yield report,yield year,york rate
0,0.793894,0.201905,0.202669,0.202755,0.201788,0.202653,0.278220,0.205862,0.233978,0.202882,...,0.201208,0.208111,0.211155,0.216487,0.227414,0.206378,0.200036,0.230241,0.239482,0.200004
1,0.200031,0.200029,0.200029,0.200030,0.200028,0.200035,0.200028,0.200033,0.200038,0.200028,...,0.200033,0.200034,0.200036,0.200035,0.200039,0.200035,0.200029,0.200038,0.200032,0.200011
2,3.224277,1.530770,2.263901,1.755749,2.835062,3.362001,5.217581,1.418039,1.507713,2.988576,...,1.698779,1.996061,1.619009,2.288248,1.571034,2.260572,2.724158,2.012869,2.309268,0.201489
3,0.200303,0.200425,0.202327,0.203029,0.200531,0.200552,0.200352,0.202186,0.200023,0.200151,...,0.200043,0.200020,0.200022,0.200144,0.200261,0.200022,0.200017,0.200022,0.200112,2.243212
4,0.200158,0.200032,0.200254,0.200213,0.200091,0.200150,0.200101,0.200097,0.200035,0.200026,...,0.202983,0.204988,0.202698,0.201984,0.201017,0.201705,0.203071,0.203893,0.200030,0.200010


## Top 10 keywords each topic

In [94]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=10)        

# Topic - Keywords Dataframe
df_keywords = pd.DataFrame(topic_keywords)
df_keywords.columns = ['Word '+str(i) for i in range(df_keywords.shape[1])]
df_keywords.index = ['Topic '+str(i) for i in range(df_keywords.shape[0])]
df_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,south dakota,north dakota,real estate,compare year,upper peninsula,paul area,resource related,increase percent,reporting period,natural resource
Topic 1,prior period,professional high,professional high tech,modest pace growth,previous survey period,wholesale trade,current period,nonauto retail,growth current period,nonauto retail sale
Topic 2,real estate,loan demand,retail sale,recent week,year ago,new order,price increase,percent year,year year,remain strong
Topic 3,new city,new jersey,upstate new,new state,northern new jersey,northern new,delinquency rate,long island,residential mortgage,vacancy rate
Topic 4,city overview,consumer lending rate,trim inventory,housing finance,material readily available,material readily,small time,deposit account,lending rate,manufacture material


## An Econoic Analysis: Applying Domain Knowledge

**Topic 0: Primary sector.** 

That is, sectors that are resource-related. Think Agriculture and Mining related activities. 

In [95]:
df_keywords.iloc[:1, :]

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,south dakota,north dakota,real estate,compare year,upper peninsula,paul area,resource related,increase percent,reporting period,natural resource


**Topic 1: Retail and Wholesale sectors.**

Wholesale trade is an economic indicator that measures the value in U.S. dollars of all merchant wholesalers' sales and inventories. Wholesale trade is one component of business sales and inventories. Only those firms that sell to governments, institutions and other businesses are considered part of wholesale trade. [See here for more details.](https://www.investopedia.com/terms/w/wholesale-trade.asp)

In [96]:
df_keywords.iloc[1:2, :]

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 1,prior period,professional high,professional high tech,modest pace growth,previous survey period,wholesale trade,current period,nonauto retail,growth current period,nonauto retail sale


**Topic 2: Consumer Spending and Business Conditions**  

[Consumer spending](https://www.bea.gov/resources/learning-center/what-to-know-consumer-spending) is a key component of quarterly gross domestic product. Statistics on consumer spending are closely followed by market participants as an initial gauge of the economy's strength. Spending data at the national and state levels help businesses predict consumer behavior and make decisions about hiring and investing. They also help policymakers understand the nexus between Americans' incomes, spending and saving.

In [97]:
df_keywords.iloc[2:3, :]

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 2,real estate,loan demand,retail sale,recent week,year ago,new order,price increase,percent year,year year,remain strong


**Topic 3: Delinquency Rate**

Delinquency rate refers to the percentage of loans within a financial institution's loan portfolio whose payments are delinquent. When analyzing and investing in loans, the delinquency rate is an important metric to follow.

The States of New Jersey and New York had the highest morgage delinquency rates in the US, according to [Creditsesame](https://www.creditsesame.com/blog/debt/states-that-dont-pay-bill-on-time/) as at 2018.

In [98]:
df_keywords.iloc[3:4, :]

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 3,new city,new jersey,upstate new,new state,northern new jersey,northern new,delinquency rate,long island,residential mortgage,vacancy rate


**Topic 4: Small Businesses**

Small businesses contribute to local economies by bringing growth and innovation to the community in which the business is established. Small businesses also help stimulate economic growth by providing employment opportunities to people who may not be employable by larger corporations.

In [99]:
df_keywords.iloc[4:5, :]

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 4,city overview,consumer lending rate,trim inventory,housing finance,material readily available,material readily,small time,deposit account,lending rate,manufacture material
